In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random

import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
#from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


2022-05-24 04:07:06.372726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 04:07:06.372759: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dataframe = pd.read_csv("../datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_IoT_dataset/Train_Test_IoT_Modbus.csv")
columns = range(3,8)
df = dataframe.iloc[:, columns]


In [3]:
#NOT REAL, USING ALL THE DATASET - NORMALIZATION MUST BE DONE LOCALLY OR TAKING INTO ACCOUNT CLIENT ROW
df_norm = df.copy()
df_norm = df_norm.iloc[:, range(4)]
for cl in df_norm.columns:
    df_norm[cl].astype('float32')
    min = np.min(df_norm[cl])
    max = np.max(df_norm[cl])
    df_norm[cl] = (df_norm[cl] - min) / (max - min)

df_norm = pd.concat([df_norm, df.iloc[:,-1]], axis=1)


In [4]:
NUM_CLIENTS = 10
BATCH_SIZE = 1
client_id_colname = 'clis'

#IID or NOIID?
choices = np.random.choice(range(NUM_CLIENTS), size=len(df_norm), replace=True)

dcli = {client_id_colname: choices}
dfcli = pd.DataFrame(data=dcli)

df = pd.concat([df_norm, dfcli], axis=1)

In [5]:
# DATA AUGMENTATION
#df = df.sample(300000, replace=True)

In [6]:

#df = pd.read_csv(csv_url, na_values=("?",))

SHUFFLE_BUFFER = 1000
NUM_EPOCHS = 1
# split client id into train and test clients
client_ids = df[client_id_colname].unique()
train_client_ids = pd.DataFrame(client_ids).sample(frac=0.8).values.tolist()
test_client_ids = [[x] for x in client_ids if x not in train_client_ids]

def create_tf_dataset_for_client_fn(client_id):
  # a function which takes a client_id and returns a
  # tf.data.Dataset for that client
  client_data = df[df[client_id_colname] == client_id[0]]
  dataset = tf.data.Dataset.from_tensor_slices(client_data.fillna('').to_dict("list"))
  dataset = dataset.shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).repeat(NUM_EPOCHS)
  return dataset

train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=train_client_ids,
        serializable_dataset_fn = create_tf_dataset_for_client_fn
        #create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )

test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
         client_ids=test_client_ids,
         serializable_dataset_fn = create_tf_dataset_for_client_fn
         #create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
     )

example_dataset = train_data.create_tf_dataset_for_client(
        train_data.client_ids[0]
    )

print(type(example_dataset))
example_element = iter(example_dataset).next()
print(example_element)


<class 'tensorflow.python.data.ops.dataset_ops.RepeatDataset'>
{'FC1_Read_Input_Register': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.23794772], dtype=float32)>, 'FC2_Read_Discrete_Value': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.95381093], dtype=float32)>, 'FC3_Read_Holding_Register': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.99769557], dtype=float32)>, 'FC4_Read_Coil': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.41250342], dtype=float32)>, 'label': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, 'clis': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>}


2022-05-24 04:07:10.132772: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-24 04:07:10.132806: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-24 04:07:10.132824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-05-24 04:07:10.133065: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
def preprocess(dataset):
  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape([element['FC1_Read_Input_Register'], element['FC2_Read_Discrete_Value'], element['FC3_Read_Holding_Register'], element['FC4_Read_Coil']], [-1, 4]),
            tf.reshape(element['label'], [-1, 1]))
  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

In [8]:
client_ids = sorted(train_data.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(train_data.create_tf_dataset_for_client(x))
  for x in client_ids
]

Preparing the model

In [9]:
#def create_keras_model():
#  initializer = tf.keras.initializers.GlorotNormal(seed=0)
#  return tf.keras.models.Sequential([
#      tf.keras.layers.Input(shape=(4,)),
#      tf.keras.layers.Dense(8),
#      tf.keras.layers.Dense(10),
#      tf.keras.layers.Dense(8),
#      #tf.keras.layers.Dense(10, kernel_initializer=initializer),
#      tf.keras.layers.Softmax(),
#  ])


In [10]:
def create_keras_model():
  #initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(4,)),
      #tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(16, activation="relu"),
      tf.keras.layers.Dense(8, activation="relu"),
      tf.keras.layers.Dense(16, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      #tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Softmax()])

In [11]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

In [12]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

In [13]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

In [14]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights


In [15]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights


## TFF

In [16]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables


In [17]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)


In [18]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

In [19]:
str(tf_dataset_type)

'<float32[?,4],int32[?,1]>*'

In [20]:
model_weights_type = server_init.type_signature.result

In [21]:
str(model_weights_type)

'<float32[4,64],float32[64],float32[64,32],float32[32],float32[32,16],float32[16],float32[16,8],float32[8],float32[8,16],float32[16],float32[16,32],float32[32],float32[32,64],float32[64]>'

In [22]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)


In [23]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)


In [24]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)


In [25]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))

  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights


In [26]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)


In [27]:
str(federated_algorithm.initialize.type_signature)


'( -> <float32[4,64],float32[64],float32[64,32],float32[32],float32[32,16],float32[16],float32[16,8],float32[8],float32[8,16],float32[16],float32[16,32],float32[32],float32[32,64],float32[64]>@SERVER)'

In [28]:
str(federated_algorithm.next.type_signature)


'(<server_weights=<float32[4,64],float32[64],float32[64,32],float32[32],float32[32,16],float32[16],float32[16,8],float32[8],float32[8,16],float32[16],float32[16,32],float32[32],float32[32,64],float32[64]>@SERVER,federated_dataset={<float32[?,4],int32[?,1]>*}@CLIENTS> -> <float32[4,64],float32[64],float32[64,32],float32[32],float32[32,16],float32[16],float32[16,8],float32[8],float32[8,16],float32[16],float32[16,32],float32[32],float32[32,64],float32[64]>@SERVER)'

## Training

In [29]:
client_ids = sorted(test_data.client_ids)[:NUM_CLIENTS]
federated_test_data = [preprocess(test_data.create_tf_dataset_for_client(x))
  for x in client_ids
]

In [34]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      optimizer = tf.optimizers.Adam(),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]#,tf.keras.metrics.BinaryAccuracy()]  
  )
  keras_model.set_weights(server_state)

  #JUST EVALUATING IN ONE CLIENTS DATA FIX IT
  keras_model.evaluate(federated_test_data[0])


In [35]:
server_state = federated_algorithm.initialize()
evaluate(server_state)


5133/5133 [==============================] - 4s 729us/step - loss: 4.1759 - sparse_categorical_accuracy: 1.9482e-04


In [36]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)


In [37]:
evaluate(server_state)



5133/5133 [==============================] - 4s 779us/step - loss: 1.0633 - sparse_categorical_accuracy: 0.6750
